In [2]:
import numpy as np
import pandas as pd
import re

# Introduction

In this notebook, we examine publicly available data of Smillie et al (2019), Cell. The purpose is to determine compositional changes using our scCODA model in the three conditions (Healthy, Non-inflamed, Inflamed). 

We perform the following steps:
1. Load the data
2. Preprocess data

# Read the data

In [8]:
data_path = './data/'

Read meta data. 
The data were downloaded from Single Cell Portal (SCP259).
The project contains also mtx-files, but we are only interested in the metadata.

In [9]:
meta = pd.read_table(data_path + 'all.meta2.txt', sep='\t')

In [10]:
meta = meta.drop([0])

In [11]:
meta.shape

(365492, 8)

In [12]:
meta.head(10)

,NAME,Cluster,nGene,nUMI,Subject,Health,Location,Sample
1,N7.EpiA.AAACATACACACTG,TA 1,328,891,N7,Non-inflamed,Epi,N7.EpiA
2,N7.EpiA.AAACCGTGCATCAG,TA 1,257,663,N7,Non-inflamed,Epi,N7.EpiA
3,N7.EpiA.AAACGCACAATCGC,TA 2,300,639,N7,Non-inflamed,Epi,N7.EpiA
4,N7.EpiA.AAAGATCTAACCGT,Enterocyte Progenitors,250,649,N7,Non-inflamed,Epi,N7.EpiA
5,N7.EpiA.AAAGATCTAGGCGA,Enterocyte Progenitors,284,769,N7,Non-inflamed,Epi,N7.EpiA
6,N7.EpiA.AAAGCCTGCTCGAA,Enterocyte Progenitors,339,951,N7,Non-inflamed,Epi,N7.EpiA
7,N7.EpiA.AAATCAACATCACG,TA 1,262,600,N7,Non-inflamed,Epi,N7.EpiA
8,N7.EpiA.AAATCAACCTTGGA,Immature Goblet,308,976,N7,Non-inflamed,Epi,N7.EpiA
9,N7.EpiA.AAATCATGGAAAGT,Enterocyte Progenitors,316,934,N7,Non-inflamed,Epi,N7.EpiA
10,N7.EpiA.AAATCCCTCACTTT,Enterocyte Progenitors,267,655,N7,Non-inflamed,Epi,N7.EpiA


Set index to "NAME" column.

In [13]:
meta.index = meta['NAME']

In [14]:
meta = meta.drop(columns = ['NAME'])

In [15]:
meta.head(10)

,Cluster,nGene,nUMI,Subject,Health,Location,Sample
NAME,,,,,,,
N7.EpiA.AAACATACACACTG,TA 1,328,891,N7,Non-inflamed,Epi,N7.EpiA
N7.EpiA.AAACCGTGCATCAG,TA 1,257,663,N7,Non-inflamed,Epi,N7.EpiA
N7.EpiA.AAACGCACAATCGC,TA 2,300,639,N7,Non-inflamed,Epi,N7.EpiA
N7.EpiA.AAAGATCTAACCGT,Enterocyte Progenitors,250,649,N7,Non-inflamed,Epi,N7.EpiA
N7.EpiA.AAAGATCTAGGCGA,Enterocyte Progenitors,284,769,N7,Non-inflamed,Epi,N7.EpiA
N7.EpiA.AAAGCCTGCTCGAA,Enterocyte Progenitors,339,951,N7,Non-inflamed,Epi,N7.EpiA
N7.EpiA.AAATCAACATCACG,TA 1,262,600,N7,Non-inflamed,Epi,N7.EpiA
N7.EpiA.AAATCAACCTTGGA,Immature Goblet,308,976,N7,Non-inflamed,Epi,N7.EpiA
N7.EpiA.AAATCATGGAAAGT,Enterocyte Progenitors,316,934,N7,Non-inflamed,Epi,N7.EpiA


# Adjust metadata according to manuscript

In [16]:
meta['Sample'].value_counts()

N58.LPB1      16723
N111.LPB1     13175
N661.LPA2     12345
N661.LPA1     12253
N661.LPB1      9450
              ...  
N106.EpiA       135
N52.EpiA2b      109
N52.EpiA2a      108
N58.EpiB2        33
N49.EpiA         21
Name: Sample, Length: 133, dtype: int64

In [17]:
meta['Location'].value_counts()

LP     266286
Epi     99206
Name: Location, dtype: int64

In [18]:
meta['Health'].value_counts()

Non-inflamed    130263
Inflamed        125119
Healthy         110110
Name: Health, dtype: int64

In [19]:
len(np.unique(meta['Subject']))

30

In [20]:
pd.crosstab(meta['Subject'], meta['Health'])

Health,Healthy,Inflamed,Non-inflamed
Subject,,,
N10,16643,0,0
N106,0,4848,2694
N11,6799,0,0
N110,0,3834,6570
N111,0,19648,5738
N12,0,1355,1009
N13,4695,0,0
N14,0,2276,2676
N15,10649,0,0


In [21]:
health = meta[['Health', 'Subject', 'Sample']]

In [22]:
replicates = pd.DataFrame([sample.split('.') for sample in np.unique(meta['Sample'])], 
                          columns=['Subject', 'Sample'], index= np.unique(meta['Sample']) )

In [23]:
replicates['Location']= [re.split('A|B',region)[0] for region in replicates['Sample']]
replicates['Replicate'] = [re.split('Epi|LP',region)[1] for region in replicates['Sample']]

In [24]:
replicates

,Subject,Sample,Location,Replicate
N10.EpiA,N10,EpiA,Epi,A
N10.EpiB,N10,EpiB,Epi,B
N10.LPA,N10,LPA,LP,A
N10.LPB,N10,LPB,LP,B
N106.EpiA,N106,EpiA,Epi,A
...,...,...,...,...
N8.LPB,N8,LPB,LP,B
N9.EpiA,N9,EpiA,Epi,A
N9.EpiB,N9,EpiB,Epi,B
N9.LPA,N9,LPA,LP,A


Merge health status info and replicate info.

In [25]:
new_meta = replicates.merge(health, how='outer', 
                            left_index=True, right_on='Sample', suffixes=('', '_y'))

In [26]:
new_meta = new_meta.drop(columns = ['Subject_y', 'Sample_y'])

Examine the different numbers of repicates per sample.

In [34]:
replicates['Subject'].value_counts()

N52     12
N58      8
N111     7
N44      4
N21      4
N13      4
N18      4
N8       4
N10      4
N9       4
N20      4
N24      4
N50      4
N11      4
N51      4
N14      4
N7       4
N46      4
N17      4
N106     4
N19      4
N16      4
N26      4
N23      4
N15      4
N661     4
N539     4
N12      4
N110     3
N49      3
Name: Subject, dtype: int64

In [27]:
new_meta.loc[new_meta['Subject']=='N52'].drop_duplicates()

,Subject,Sample,Location,Replicate,Health
NAME,,,,,
N52.EpiA1a.AAACCTGAGACACTAA,N52,EpiA1a,Epi,A1a,Non-inflamed
N52.EpiA1b.AAACCTGAGACACTAA,N52,EpiA1b,Epi,A1b,Non-inflamed
N52.EpiA2a.AAAGATGTCAATAAGG,N52,EpiA2a,Epi,A2a,Non-inflamed
N52.EpiA2b.AAAGATGTCAATAAGG,N52,EpiA2b,Epi,A2b,Non-inflamed
N52.LPA1a.AAACCTGCACACTGCG,N52,LPA1a,LP,A1a,Non-inflamed
N52.LPA1b.AAACCTGAGCTTTGGT,N52,LPA1b,LP,A1b,Non-inflamed
N52.LPA2a.AAACCTGCAAGTTGTC,N52,LPA2a,LP,A2a,Non-inflamed
N52.LPA2b.AAACCTGCAAGTTGTC,N52,LPA2b,LP,A2b,Non-inflamed
N52.LPB1a.AAAGCAAGTGTCCTCT,N52,LPB1a,LP,B1a,Inflamed


In [28]:
new_meta.loc[new_meta['Subject']=='N58'].drop_duplicates()

,Subject,Sample,Location,Replicate,Health
NAME,,,,,
N58.EpiA1.AAACCTGGTCGACTGC,N58,EpiA1,Epi,A1,Non-inflamed
N58.EpiA2.AAACCTGAGTATCTCG,N58,EpiA2,Epi,A2,Non-inflamed
N58.EpiB1.AAACCTGCATCGGGTC,N58,EpiB1,Epi,B1,Inflamed
N58.EpiB2.AAAGCAAGTCCAAGTT,N58,EpiB2,Epi,B2,Inflamed
N58.LPA1.AAACCTGTCTGGGCCA,N58,LPA1,LP,A1,Non-inflamed
N58.LPA2.AAACCTGCAATCAGAA,N58,LPA2,LP,A2,Non-inflamed
N58.LPB1.AAACCTGCAGGTCTCG,N58,LPB1,LP,B1,Inflamed
N58.LPB2.CCTCTGAAGGATATAC,N58,LPB2,LP,B2,Inflamed


In [29]:
new_meta.loc[new_meta['Subject']=='N111'].drop_duplicates()

,Subject,Sample,Location,Replicate,Health
NAME,,,,,
N111.EpiA1.AAACCTGGTTGAGGTG,N111,EpiA1,Epi,A1,Non-inflamed
N111.EpiB1.AAACGGGGTTCACGGC,N111,EpiB1,Epi,B1,Inflamed
N111.EpiB2.AAACCTGCAGGCAGTA,N111,EpiB2,Epi,B2,Inflamed
N111.LPA1.AAACGGGTCGCGGATC,N111,LPA1,LP,A1,Non-inflamed
N111.LPA2.AAACCTGTCCGCAAGC,N111,LPA2,LP,A2,Non-inflamed
N111.LPB1.AAAGATGGTGTCTGAT,N111,LPB1,LP,B1,Inflamed
N111.LPB2.AAACGGGGTGGCGAAT,N111,LPB2,LP,B2,Inflamed


In [30]:
new_meta.loc[new_meta['Subject']=='N110'].drop_duplicates()

,Subject,Sample,Location,Replicate,Health
NAME,,,,,
N110.EpiA.AAACCTGAGGACTGGT,N110,EpiA,Epi,A,Non-inflamed
N110.LPA.AAACCTGAGCCACTAT,N110,LPA,LP,A,Non-inflamed
N110.LPB.AAACCTGAGGACGAAA,N110,LPB,LP,B,Inflamed


In [31]:
new_meta.loc[new_meta['Subject']=='N49'].drop_duplicates()

,Subject,Sample,Location,Replicate,Health
NAME,,,,,
N49.EpiA.AAACGGGGTTCTGAAC,N49,EpiA,Epi,A,Non-inflamed
N49.LPA.AAACGGGCATGAAGTA,N49,LPA,LP,A,Non-inflamed
N49.LPB.AAACCTGTCAACACGT,N49,LPB,LP,B,Inflamed


In [32]:
new_meta.loc[new_meta['Subject']=='N19'].drop_duplicates()

,Subject,Sample,Location,Replicate,Health
NAME,,,,,
N19.EpiA.AAACATTGCAGCTA,N19,EpiA,Epi,A,Non-inflamed
N19.EpiB.AAACATTGAGTACC,N19,EpiB,Epi,B,Inflamed
N19.LPA.AAACGCTGGTAGGG,N19,LPA,LP,A,Non-inflamed
N19.LPB.AAACGCACATCGGT,N19,LPB,LP,B,Inflamed


In [33]:
new_meta.loc[new_meta['Subject']=='N8'].drop_duplicates()

,Subject,Sample,Location,Replicate,Health
NAME,,,,,
N8.EpiA.AAACATTGCGTTGA,N8,EpiA,Epi,A,Healthy
N8.EpiB.AAAGTTTGACCCTC,N8,EpiB,Epi,B,Healthy
N8.LPA.AACAATACAGTGTC,N8,LPA,LP,A,Healthy
N8.LPB.AACTCGGATGTCCC,N8,LPB,LP,B,Healthy


Check the number of cell types.

In [35]:
len(np.unique(meta['Cluster']))

51

Merge health status info and replicate info with the remaining metadata.

In [36]:
meta = new_meta.merge(meta, how='outer', left_index=True, right_index=True, suffixes=('', '_y'))

In [37]:
meta

,Subject,Sample,Location,Replicate,Health,Cluster,nGene,nUMI,Subject_y,Health_y,Location_y,Sample_y
NAME,,,,,,,,,,,,
N10.EpiA.AAACATACAACCAC,N10,EpiA,Epi,A,Healthy,Enterocyte Progenitors,425,968,N10,Healthy,Epi,N10.EpiA
N10.EpiA.AAACATACAGGCGA,N10,EpiA,Epi,A,Healthy,Cycling TA,1695,7273,N10,Healthy,Epi,N10.EpiA
N10.EpiA.AAACATACCACTAG,N10,EpiA,Epi,A,Healthy,Immature Goblet,391,1190,N10,Healthy,Epi,N10.EpiA
N10.EpiA.AAACATACCCTTTA,N10,EpiA,Epi,A,Healthy,Secretory TA,1327,5620,N10,Healthy,Epi,N10.EpiA
N10.EpiA.AAACATACTGCAAC,N10,EpiA,Epi,A,Healthy,Immature Enterocytes 2,1383,4676,N10,Healthy,Epi,N10.EpiA
...,...,...,...,...,...,...,...,...,...,...,...,...
N9.LPB.TTTATCCTAACGAA,N9,LPB,LP,B,Inflamed,Enterocytes,2768,18811,N9,Inflamed,LP,N9.LPB
N9.LPB.TTTATCCTGTAAAG,N9,LPB,LP,B,Inflamed,Plasma,1392,27685,N9,Inflamed,LP,N9.LPB
N9.LPB.TTTATCCTGTCGTA,N9,LPB,LP,B,Inflamed,Plasma,574,5478,N9,Inflamed,LP,N9.LPB


Drop duplicate columns.

In [38]:
meta = meta.drop(columns = [full_name for full_name in meta.columns if full_name.endswith('_y')])

## Merge several clusters to test for differences on a high level

In Supplementary Figure 1B of Smillie et al. (2019), the authors display a rough cell type annotation into fibroblast, endothelial, glia, myeloid, T cells, NKs/ILCs and B cells. We follow their annotation to reduce the number of classes. Of note, figure 1B seems to be contradictory in claiming to display "major cell lineages", because the grouping of cell types is different in other figures, e.g. S1A, Figure 1 in the main text (or they left out the Epithelial cells). 
The following assignment were extracted from Supplementary Figure 1D:

- Fibroblasts: WNT2B+ Fos-hi, WNT2B+ Fos-lo 1,WNT2B+ Fos-lo 2, WNT5B+ 1, WNT5B+ 2, RSPO3+, Inflammatory Fibroblasts, Myofibroblasts, Endothelial, Microvascular, Post-capillary Venules, Pericytes, Glia
- Epithelial: Stem, TA 1, TA 2, Cycling TA, Immature Enterocytes 1, Immature Enterocytes 2, Enterocytes, M-like cells, Best4+ Enterocytes, Secretory TA, Immature Goblet, Goblet, Tuft, Enteroendocrine, Enterocyte Progenitors
- Immune: Macrophages, DC1, DC2, Inflammatory Monocytes, CD69+ Mast, CD69- Mast, Cycling Monocytes, CD4+ Activated Fos-hi, CD4+ Activated Fos-lo, CD4+ Memory, NKs, ILCs, Tregs, CD4+ PD1+, CD8+ IELs, CD8+ IL17+, CD8+ LP, MT-hi, Cycling T, Plasma, Follicular, GC, Cycling B

The rough annotation in Figure 1C can be inferred as follows:
- Fibroblasts: WNT2B+ Fos-hi, WNT2B+ Fos-lo 1,WNT2B+ Fos-lo 2, WNT5B+ 1, WNT5B+ 2, RSPO3+, Inflammatory Fibroblasts, Myofibroblasts, 
- Endothelial: Endothelial, Microvascular, Post-capillary Venules, Pericytes 
- Glia
- NKs/ILCs: NKs, ILCs
- Myeloid: Macrophages, DC1, DC2, Inflammatory Monocytes, CD69+ Mast, CD69- Mast, Cycling Monocytes
- T cells: CD4+ Activated Fos-hi, CD4+ Activated Fos-lo, CD4+ Memory, Tregs, CD4+ PD1+, CD8+ IELs, CD8+ IL17+, CD8+ LP, MT-hi, Cycling T
- B cells: Plasma, Follicular, GC, Cycling B
- Epithelial: Stem, TA 1, TA 2, Cycling TA, Immature Enterocytes 1, Immature Enterocytes 2, Enterocytes, M-like cells, Best4+ Enterocytes, Secretory TA, Immature Goblet, Goblet, Tuft, Enteroendocrine, Enterocyte Progenitors

In [39]:
meta['Cluster'] = meta['Cluster'].astype('category')

In [40]:
meta['Major_l1'] = meta['Cluster'].cat.add_categories(['Fibroblasts', 'Epithelial', 'Immune'])

In [41]:
meta['Major_l1'][np.in1d(meta['Major_l1'], ['WNT2B+ Fos-hi', 'WNT2B+ Fos-lo 1', 'WNT2B+ Fos-lo 2', 'WNT5B+ 1', 'WNT5B+ 2', 'RSPO3+', 
                                      'Inflammatory Fibroblasts', 'Myofibroblasts', 'Endothelial', 'Microvascular', 
                                      'Post-capillary Venules', 'Pericytes', 'Glia'])] = 'Fibroblasts'
meta['Major_l1'][np.in1d(meta['Major_l1'], ['Stem', 'TA 1', 'TA 2', 'Cycling TA', 'Immature Enterocytes 1', 'Immature Enterocytes 2', 
                         'Enterocytes', 'M cells', 'Best4+ Enterocytes', 'Secretory TA', 'Immature Goblet', 'Goblet', 
                         'Tuft', 'Enteroendocrine', 'Enterocyte Progenitors'])] = 'Epithelial'
meta['Major_l1'][np.in1d(meta['Major_l1'], ['Macrophages', 'DC1', 'DC2', 'Inflammatory Monocytes', 'CD69+ Mast', 'CD69- Mast', 
                                      'Cycling Monocytes', 'CD4+ Activated Fos-hi', 'CD4+ Activated Fos-lo', 'CD4+ Memory', 
                                      'NKs', 'ILCs', 'Tregs', 'CD4+ PD1+', 'CD8+ IELs', 'CD8+ IL17+', 'CD8+ LP', 'MT-hi', 
                                      'Cycling T', 'Plasma', 'Follicular', 'GC', 'Cycling B'])] = 'Immune'
meta['Major_l1'] = meta['Major_l1'].cat.remove_unused_categories()

In [42]:
meta['Major_l2'] = meta['Cluster'].cat.add_categories(['Fibroblasts', 'Epithelial', 'Myeloid', 'T cells', 'B cells', 'NKs/ILCs'])

In [43]:
meta['Major_l2'][np.in1d(meta['Major_l2'], ['WNT2B+ Fos-hi', 'WNT2B+ Fos-lo 1', 'WNT2B+ Fos-lo 2', 'WNT5B+ 1', 'WNT5B+ 2', 'RSPO3+', 
                                      'Inflammatory Fibroblasts', 'Myofibroblasts'])] = 'Fibroblasts'
meta['Major_l2'][np.in1d(meta['Major_l2'], ['Endothelial', 'Microvascular', 
                                      'Post-capillary Venules', 'Pericytes'])] = 'Endothelial'
meta['Major_l2'][np.in1d(meta['Major_l2'], ['Stem', 'TA 1', 'TA 2', 'Cycling TA', 'Immature Enterocytes 1', 'Immature Enterocytes 2', 
                         'Enterocytes', 'M cells', 'Best4+ Enterocytes', 'Secretory TA', 'Immature Goblet', 'Goblet', 
                         'Tuft', 'Enteroendocrine', 'Enterocyte Progenitors'])] = 'Epithelial'
meta['Major_l2'][np.in1d(meta['Major_l2'], ['Plasma', 'Follicular', 'GC', 'Cycling B'])] = 'B cells'
meta['Major_l2'][np.in1d(meta['Major_l2'], ['Macrophages', 'DC1', 'DC2', 'Inflammatory Monocytes', 'CD69+ Mast', 'CD69- Mast', 
                                      'Cycling Monocytes'])] = 'Myeloid'
meta['Major_l2'][np.in1d(meta['Major_l2'], [ 'CD4+ Activated Fos-hi', 'CD4+ Activated Fos-lo', 'CD4+ Memory', 
                                      'NKs', 'ILCs', 'Tregs', 'CD4+ PD1+', 'CD8+ IELs', 'CD8+ IL17+', 'CD8+ LP', 'MT-hi', 
                                      'Cycling T'])] = 'T cells'
meta['Major_l2'] = meta['Major_l2'].cat.remove_unused_categories()

In [44]:
pd.value_counts(meta['Major_l1'])

Immune         210614
Epithelial     123006
Fibroblasts     31872
Name: Major_l1, dtype: int64

In [45]:
pd.value_counts(meta['Major_l2'])

Epithelial     123006
B cells        107246
T cells         76058
Myeloid         27310
Fibroblasts     24290
Endothelial      6320
Glia             1262
Name: Major_l2, dtype: int64

Convert `Major_l2` from categorical to string.

In [46]:
meta['Major_l2'] = meta['Major_l2'].astype(str)

Save to file.

In [50]:
meta.to_csv(data_path + 'meta_processed.csv')